DEFINE "PATH_TO_IMAGES" on your local machine

In [1]:
import sys
import os

import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pprint

from warnings import filterwarnings
filterwarnings("ignore")

sys.path.append(os.path.join(str(os.path.abspath('')), "..", "..", "..", "..", ".."))

from src.classification.resNet_model import CovidResNetModel
from src.classification.efficientNet_model import CovidEffnetModel
from src.classification.generalized_clf import CovidGeneralModel
from src.classification.train_one_epoch import train_one_epoch
from src.classification.validate_func import valid_func
from src.classification.classification_transforms import get_transform
from src.classification.get_data_loaders import prepare_train_valid_dataloader

In [2]:
class CFG:
    debug = False # change this to run on full data
    
    image_size = 384
    target_columns = [
        'Negative for Pneumonia',
        'Typical Appearance',
        'Indeterminate Appearance',
        'Atypical Appearance'
    ]
    folds = [0, 1, 2, 3, 4]
    
    kernel_type = 'vit_base_r50_s16_384'
    train_batch_size = 10
    valid_batch_size = 24
    
    init_lr = 1e-4
    weight_decay=1e-5
    
    n_epochs = 20
    num_workers = 4

    use_amp=False
    early_stop = 5

    data_dir = PATH_TO_IMAGES # !!! DEFINE "PATH_TO_IMAGES" on your local machine
    model_dir = f'weights/'
    seed=42
    

In [3]:
! mkdir $CFG.model_dir

mkdir: cannot create directory ‘weights/’: File exists


In [4]:
df_train = pd.read_csv('../cross_validation_scheme/crossval_info/original_data/multi_label_problem.csv')
df_train['image_name'] = df_train['image_id'].apply(lambda x: x.replace('_image', '.png'))
df_train['file_path'] = df_train.image_name.apply(lambda x: os.path.join(CFG.data_dir, f'{x}'))
if CFG.debug:
    df_train = df_train.sample(frac=0.1)

In [5]:
train_transform = get_transform(img_size=CFG.image_size, transform_type='try_new_strong')
valid_transform = get_transform(img_size=CFG.image_size, transform_type='val')

criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
for fold in CFG.folds:
    train_loader, valid_loader = prepare_train_valid_dataloader(
        df=df_train, fold=fold, target_cols=CFG.target_columns,
        train_transform=train_transform, valid_transform=valid_transform,
        train_batch_size=CFG.train_batch_size, valid_batch_size=CFG.valid_batch_size,
        num_workers=CFG.num_workers
    )
    
#     model = CovidEffnetModel(model_name=CFG.kernel_type, pretrained=True).to(device)
    model = CovidGeneralModel(model_name=CFG.kernel_type, pretrained=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=CFG.init_lr, weight_decay=CFG.weight_decay)
    scheduler = ReduceLROnPlateau(
        optimizer, mode='min', patience=1, min_lr=1e-8, factor=0.1, verbose=True, eps=1e-8
    )

    num_epochs = CFG.n_epochs

    print("-----------------------------------------------------------------------------------------------------")
    print("                                        FOLD: ", fold)
    print("-----------------------------------------------------------------------------------------------------")
    
    roc_auc_max = 0.0
    loss_min = 99999
    ap_max = 0.0
    not_improving = 0
    metrics_list = list()
    
    for epoch in range(num_epochs):
        
        loss_train, roc_auc_train, average_precision_train = train_one_epoch(
            model, device, criterion, optimizer, train_loader, CFG.use_amp, CFG.target_columns)
        
        loss_valid, roc_auc_valid, average_precision_valid = valid_func(
            model, device, criterion, valid_loader, CFG.target_columns)
        
        scheduler.step(loss_valid)
        
        metrics_dictionary = {}
        metrics_dictionary['epoch'] = epoch
        metrics_dictionary['loss_train'] = loss_train
        metrics_dictionary['loss_valid'] = loss_valid
        metrics_dictionary['roc_auc_train'] = roc_auc_train
        metrics_dictionary['roc_auc_valid'] = roc_auc_valid
        metrics_dictionary['average_precision_train'] = average_precision_train
        metrics_dictionary['average_precision_valid'] = average_precision_valid
        metrics_dictionary['fold'] = fold
        pprint.pprint(metrics_dictionary)
        metrics_list.append(metrics_dictionary)
        
        not_improving += 1
        if roc_auc_valid > roc_auc_max:
            print(f'roc_auc_max ({roc_auc_max:.6f} --> {roc_auc_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_best_AUC.pth')
            roc_auc_max = roc_auc_valid
            not_improving = 0

        if loss_valid < loss_min:
            loss_min = loss_valid
            torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_best_loss.pth')

        if average_precision_valid > ap_max:
            print(f'ap_max ({ap_max:.6f} --> {average_precision_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_best_AP.pth')
            ap_max = average_precision_valid
#             not_improving = 0
            
        if not_improving == CFG.early_stop:
            print('Early Stopping...')
            break

    
    metrics = pd.DataFrame(metrics_list)
    metrics.to_csv(f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_final.csv', index=False)
    torch.save(model.state_dict(), f'{CFG.model_dir}{CFG.kernel_type}_fold{fold}_final.pth')


  0%|          | 0/507 [00:00<?, ?it/s]

-----------------------------------------------------------------------------------------------------
                                        FOLD:  0
-----------------------------------------------------------------------------------------------------


loss: 0.48044, smth: 0.47661: 100%|██████████| 53/53 [00:11<00:00,  4.45it/s]


{'average_precision_train': 0.2662572567833547,
 'average_precision_valid': 0.3167014881759896,
 'epoch': 0,
 'fold': 0,
 'loss_train': 0.44913920842449107,
 'loss_valid': 0.47281023297669755,
 'roc_auc_train': 0.6646755515949697,
 'roc_auc_valid': 0.7449565409694058}
roc_auc_max (0.000000 --> 0.744957). Saving model ...
ap_max (0.000000 --> 0.316701). Saving model ...


loss: 0.45673, smth: 0.40397: 100%|██████████| 53/53 [00:11<00:00,  4.49it/s]


...

Epoch    18: reducing learning rate of group 0 to 1.0000e-07.
{'average_precision_train': 0.3862647875501833,
 'average_precision_valid': 0.3781785619438027,
 'epoch': 17,
 'fold': 4,
 'loss_train': 0.3508547919506622,
 'loss_valid': 0.3661263084636544,
 'roc_auc_train': 0.8257697669581554,
 'roc_auc_valid': 0.8117691709157013}
roc_auc_max (0.810806 --> 0.811769). Saving model ...
ap_max (0.377367 --> 0.378179). Saving model ...


loss: 0.56801, smth: 0.37711: 100%|██████████| 53/53 [00:11<00:00,  4.54it/s]


{'average_precision_train': 0.38658269895245706,
 'average_precision_valid': 0.37824839243212827,
 'epoch': 18,
 'fold': 4,
 'loss_train': 0.3488562889911484,
 'loss_valid': 0.36623518000233846,
 'roc_auc_train': 0.823942537122051,
 'roc_auc_valid': 0.8117198557511925}
ap_max (0.378179 --> 0.378248). Saving model ...


loss: 0.56793, smth: 0.37705: 100%|██████████| 53/53 [00:11<00:00,  4.54it/s]


Epoch    20: reducing learning rate of group 0 to 1.0000e-08.
{'average_precision_train': 0.3835330991767607,
 'average_precision_valid': 0.37836956887227396,
 'epoch': 19,
 'fold': 4,
 'loss_train': 0.35039714625366103,
 'loss_valid': 0.36616249494957476,
 'roc_auc_train': 0.8243803567202295,
 'roc_auc_valid': 0.8118128752215996}
roc_auc_max (0.811769 --> 0.811813). Saving model ...
ap_max (0.378248 --> 0.378370). Saving model ...
